In [11]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

# Decryption function
def decode_cf_email(encoded_email):
    decoded_email = ""
    k = int(encoded_email[:2], 16)

    for i in range(2, len(encoded_email), 2):
        decoded_email += chr(int(encoded_email[i:i+2], 16) ^ k)

    return decoded_email

# Function to fetch company emails
def fetch_company_emails():
    base_url = "https://www.teknoparkistanbul.com.tr"
    url = "https://www.teknoparkistanbul.com.tr/firmalar"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # Fetch HTML content of the main page
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve the webpage: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all company links
    company_links = soup.find_all('a', href=re.compile(r'/firmalar/[\w-]+'))

    company_emails = []
    count = 0
    for link in company_links:
        company_relative_url = link['href']
        company_url = urljoin(base_url, company_relative_url)  # Correctly construct the company URL

        try:
            # Fetch HTML content of the company page
            company_response = requests.get(company_url, headers=headers)
            company_response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve company page {company_url}: {e}")
            continue

        company_soup = BeautifulSoup(company_response.content, 'html.parser')

        # Find all email elements containing encoded emails
        email_elements = company_soup.find_all('span', class_='__cf_email__')

        count = count + 1
        for email_element in email_elements:
            encoded_email = email_element['data-cfemail']  # Extract the encoded part
            decoded_email = decode_cf_email(encoded_email)
            company_emails.append(decoded_email)
            print(decoded_email)
    print(f"{count} tane email alindi")

    # Remove duplicates
    company_emails = list(set(company_emails))

    # Write emails to a text file
    with open('company_emails.txt', 'w') as file:
        for email in company_emails:
            file.write(email + '\n')

    print(f"Successfully extracted {len(company_emails)} email addresses.")

# Execute the function
fetch_company_emails()


acarmant@gmail.com
info@a2.tc
info@abramakbilisim.com
info.IndustryXTR@accenture.com
tr-office@acronis.com
info@adente.com.tr
info@advens.com.tr
info@agenabst.com
info@aimpms.com
info@aisenseus.net
info@akkoctech.com
info@albarakatech.com
info@alloyadditive.com
savunma@altinay.com
info@draminoacid.co.uk
info@anayurtteknoloji.com
info@andasis.com
osak@dogus.edu.tr
info@aotmuhendislik.com
info@aras-marine.com
info@aras-marine.com
info@architecht.com
info@ardifarma.com
evren.gunay@aridrones.com
info@armelsan.com
aaras@istanbul.edu.tr
info@arti.tc
info@arventek.com
info@arvis.com.tr
info@asartech.com.tr
info@aselsan.com.tr
info@asistbt.com.tr
info@asfat.com.tr
aspilsan@aspilsan.com.tr
info@atasayin.com
ahmet@atlantisotomasyon.com
info@avikon.com.tr
info@avitek.com
info@ayeminovation.com
info@aykan.com
info@ayrotec.com
azargest@hotmail.com
contact@b2metric.com
info@bakelor.com
info@basebros.com
bilgi@basf.com
info@berkinengineering.com
info@bimteknoloji.com.tr
info@bio-t.org
bilgi@bisatek.i

In [13]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("gmail")
secret_value_1 = user_secrets.get_secret("gmail_sifre")

In [14]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
import os

def send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path):
    # Establishing connection with Gmail's SMTP server
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(sender_email, sender_password)
    
    # Creating email message
    message = MIMEMultipart()
    message['From'] = "Yusuf Kemal Pınarcı"
    message['To'] = receiver_email
    message['Subject'] = subject
    
    # Attaching body to the email
    message.attach(MIMEText(body, 'plain'))
    
    # Attaching the resume as an attachment
    filename = os.path.basename(attachment_path)
    attachment = open(attachment_path, "rb")
    
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f'attachment; filename= {filename}')
    
    message.attach(part)
    text = message.as_string()
    
    # Sending the email
    server.sendmail(sender_email, receiver_email, text)
    
    # Closing the connection
    server.quit()

# Example usage:
if __name__ == "__main__":
    sender_email = secret_value_0
    sender_password = secret_value_1
    receiver_email = "cubukcums@gmail.com"
    subject = "İş Başvuru"
    body = """Sayın İnsan Kaynakları Yetkilisi,

Konya Teknik Üniversitesi Bilgisayar Mühendisliği bölümünden yeni mezun oldum.Yazılım dünyasında web, mobil uygulama geliştirme ve yapay zeka ile ilgileniyorum. 
Bu alanda çeşitli projeler üzerinde aktif olarak çalışıyor ve yeni teknolojiler öğreniyorum.

Kendimi geliştirebileceğim ve önemli işlere imza atacağım bir iş arıyorum. İletişim, analitik düşünme ve takım çalışması becerilerimle, şirketinize katkı sağlayabileceğime inanıyorum.

CV'mi ekte bulabilirsiniz. Uygun pozisyonlarınız olduğunda benimle iletişime geçmenizi umuyorum.
İlginiz ve zamanınız için teşekkür ederim.

Saygılarımla,

Yusuf Kemal Pınarcı
+90 551 955 6453"""
    attachment_path = "/kaggle/input/my-resume/cv_Yusuf_Kemal_Pinarci.pdf"  # Replace with your resume file path
    
    send_email(sender_email, sender_password, receiver_email, subject, body, attachment_path)


In [18]:
sent_companies = []

In [25]:
with open('/kaggle/input/company-emails2/company_emails.txt', 'r') as file:
    for company_email in file:
        if company_email in sent_companies:
            continue
            print(f"Zaten {company_email} sirketine mail gonderildi")
        else:
            send_email(sender_email, sender_password, company_email, subject, body, attachment_path)
            sent_companies.append(company_email)
            print(f"{company_email} sirketine email basariyla gonderildi")
            


info@makroaero.com
 sirketine email basariyla gonderildi
info@atasayin.com
 sirketine email basariyla gonderildi
info@reelalacak.com
 sirketine email basariyla gonderildi
info@dert.solutions
 sirketine email basariyla gonderildi
info@marvajed.com
 sirketine email basariyla gonderildi
kurumsal@infotech.com.tr
 sirketine email basariyla gonderildi
info@digicore.com.tr
 sirketine email basariyla gonderildi
malraen3418@gmail.com
 sirketine email basariyla gonderildi
info@meccon.com.tr
 sirketine email basariyla gonderildi
info@arvis.com.tr
 sirketine email basariyla gonderildi
info@phobosrobotics.com
 sirketine email basariyla gonderildi
info@visiomex.com
 sirketine email basariyla gonderildi
info@hidro-teknik.net
 sirketine email basariyla gonderildi
support@ferrumgate.com
 sirketine email basariyla gonderildi
hulya@umayana.us
 sirketine email basariyla gonderildi
info@pavotek.com.tr
 sirketine email basariyla gonderildi
info@iradets.com
 sirketine email basariyla gonderildi
info@naviserv